# Notebook de cliente

In [ ]:
from datasets import load_dataset
from model import NERClassifier
from preprocess_dataset import NERDataset 
from trainner import Trainner
from transformers import BertTokenizer
from preprocess_dataset import remove_empty_entries

import torch

## Setups

In [ ]:
CHECKPOINT = 'neuralmind/bert-base-portuguese-cased'
tokenizer = BertTokenizer.from_pretrained(CHECKPOINT)

In [ ]:
data = "lener_br"
dataset = load_dataset(data)
dataset = remove_empty_entries(dataset)

In [ ]:
MAX_LEN = 256
n_labels = 13 #TODO: por enquanto hardcodado 

In [ ]:
pytorch_dataset_train = NERDataset(data=dataset['train'], max_len=MAX_LEN, tokenizer=tokenizer)
teste = pytorch_dataset_train[0]

In [ ]:

model = NERClassifier(n_labels=n_labels, checkpoint=CHECKPOINT)
input_ids = teste['input_ids']
attention_mask = teste['attention_mask']
labels = teste['targets']

In [ ]:
type(dataset)

In [ ]:
# dataset = remove_empty_entries(dataset)

In [ ]:
dataset

In [ ]:
max_vals = [torch.max(val['targets']) for val in pytorch_dataset_train]

In [ ]:
interval = (min(max_vals).item(), max(max_vals).item())

In [ ]:
interval

----------------------------------------

## Treinando o modelo

Usando parametros padrão por enquanto só pra ver se ta tudo indo certindo

In [ ]:
from trainner import Trainner, create_data_loader
from transformers import AdamW
from torch.utils.data import DataLoader

### Parametros


* batch sizes: 8, 16, 32, 64, 128
* learning rates: 3e-4, 1e-4, 5e-5, 3e-5

Do paper do BERT. Vou colocar num_workers = 4 por enquanto.


Uma melhoria é esconder o data_loader de clientes. Isso é detalhe de implementaçãõ, ou deveria estar na classe NERDataset

In [ ]:
optimizer = AdamW(model.parameters(), lr=3e-4, correct_bias=False)

In [ ]:
len(pytorch_dataset_train)

In [ ]:
loader = DataLoader(pytorch_dataset_train, batch_size=8)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
next(model.parameters()).is_cuda

In [ ]:
tr = Trainner(device, loader, model, optimizer)

In [ ]:
losses, media = tr._train_epoch()

In [ ]:
losses

In [ ]:
from matplotlib import pyplot as plt

In [ ]:
plt.plot(losses)